In [ ]:
import RF_input_parameters as RF_inp
import RF_tile_creator as RF_tc

RF_params = RF_inp.RF_Parameters()  # Create base parameters
RF_tc.create_tiles(RF_params)


In [2]:
if RF_params.add_classifier_params().model_path is None:
    import RF_trainer as RF_tr
    RF_tr.train_model(RF_params)

In [ ]:
import RF_tile_classifier as RF_cl
RF_cl.classify_tiles(RF_params)

In [5]:
import os
import RF_input_parameters as RF_inp
import RF_tile_creator as RF_tc

#check DEM and Ortho path location
inputs_dict =   {'UM2_071922': [r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\Input DEMs and Orthos\UM2_2023____071922_PostError_PCFiltered_Ortho_5cm.tif",
                                r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\Input DEMs and Orthos\UM2_2023____071922_PostError_PCFiltered_DEM_5cm.tif"],
                 'UM1_090822' : [r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\Input DEMs and Orthos\UM1_2023____090822_PostError_PCFiltered_Ortho_5cm.tif",
                                r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\Input DEMs and Orthos\UM1_2023____090822_PostError_PCFiltered_DEM_5cm.tif"]
}


for key, value in inputs_dict.items():
    RF_params = RF_inp.RF_Parameters()  # Create base parameters
    RF_params.DEM_path = value[1]
    RF_params.ortho_path = value[0]
    RF_params.output_folder = r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\{}".format(key)
    print(RF_params.output_folder)  
    if not os.path.exists(RF_params.output_folder):
        os.makedirs(RF_params.output_folder)
    RF_params.record_params()
    RF_tc.create_tiles(RF_params)
    import RF_tile_classifier as RF_cl
    RF_cl.classify_tiles(RF_params)



Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\UM2_071922
Grid IDs: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]
Processing grid cell 1 of 64
Skipping grid cell 0 due to lack of valid pixels in cell.
Processing grid cell 2 of 64
Grid cell 1 processed.
Output saved to Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\UM2_071922\RF_Tiled_Inputs\stacked_bands_tile_input_1.tif
Processing grid cell 3 of 64
Grid cell 2 processed.
Output saved to Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\UM2_071922\RF_Tiled_Inputs\stacked_bands_tile_input_2.tif
Processing grid cell 4 of 64
Skipping grid cell 3 due to lack of valid pixels in cell.
Processing grid cell 5 of 64
Skipping grid cell 4 due to lack of valid pixels in cell.
Processing grid cell 6 of 64
Skipping grid cell 5 due to lack o